In [ ]:
import io
import sys
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
# this stops all the website url being printed when scraping multiple webpages
text_trap = io.StringIO()
sys.stdout = text_trap

# This restore printing
# sys.stdout = sys.__stdout__ 

## Change the site ID. 
### IMPORTANT!!
### Please make sure that the [ Reviews{0} ] index is embedded in the site ID as the code scrapes multiple pages with the idx


In [ ]:
site_id = 'https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d2074022-Reviews{0}-The_Prince_of_Wales-London_England.html'

### Functions to request url and access reviews with BeautifulSoup

In [ ]:
results=list()
no_response=False

def get_url(current_url):
    file = requests.get(current_url)
    if file.status_code!=200:
        print('no response')
        no_respnse=True
    
    response = BeautifulSoup(file.text, 'html.parser')
    return [response, no_response]

def get_reviews(url, response):
    print(url)
    # get every review
    for review in response.find_all('div', class_='review-container'):
        
        if review.find(class_='mgrRspnInline'): # take out manager's reply
            continue
        elif review.find('p', class_='partial_entry'):
            append_content = review.find('p', class_='partial_entry').text
            
            if review.find(class_='bubble_50'):
                rating='5'
            elif review.find(class_='bubble_40'):
                rating='4'
            elif review.find(class_='bubble_30'):
                rating='3'
            elif review.find(class_='bubble_20'):
                rating='2'
            elif review.find(class_='bubble_10'):
                rating='1'
            
        item = {'Restaurant_name': response.find('h1').text, # Restaurant name is outside of review-container
            'review_title': review.find('span', class_='noQuotes').text,
            'review_content': append_content,
            'review_rating': rating}

        results.append(item) # <--- add to global list


### Scrape multiple pages (need to manually decide how many pages)

In [ ]:
total_pg = 76
for idx in range(0,total_pg):
    s ='' if idx==0 else '-or{0}0'.format(idx)
    site_idx = site_id.format(s)
              
    [response, no_response] = get_url(site_idx)
    if no_response == False:
        get_reviews(site_idx, response)
    else:
        break
    

### Save reviews

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('{0}.csv'.format(df['Restaurant_name'][0]))  